In [64]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

## Overview

In this notebook, we want to provide an overview what HugeCTR framework is, its features and benefits. We will use HugeCTR to train a basic neural network architecture and deploy the saved model to Triton Inference Server. 

<b>Learning Objectives</b>:
* Adopt NVTabular workflow to provide input files to HugeCTR
* Define HugeCTR neural network architecture
* Train a deep learning model with HugeCTR
* Deploy HugeCTR to Triton Inference Server

### Why using HugeCTR?

HugeCTR is a GPU-accelerated recommender framework designed to distribute training across multiple GPUs and nodes and estimate Click-Through Rates (CTRs).<br>

HugeCTR offers multiple advantages to train deep learning recommender systems:
1. **Speed**: HugeCTR is a highly efficient framework written C++. We experienced up to 10x speed up. HugeCTR on a NVIDIA DGX A100 system proved to be the fastest commercially available solution for training the architecture Deep Learning Recommender Model (DLRM) developed by Facebook.
2. **Scale**: HugeCTR supports model parallel scaling. It distributes the large embedding tables over multiple GPUs or multiple nodes. 
3. **Easy-to-use**: Easy-to-use Python API similar to Keras. Examples for popular deep learning recommender systems architectures (Wide&Deep, DLRM, DCN, DeepFM) are available.

### Other Features of HugeCTR

HugeCTR is designed to scale deep learning models for recommender systems. It provides a list of other important features:
* Proficiency in oversubscribing models to train embedding tables with single nodes that don’t fit within the GPU or CPU memory (only required embeddings are prefetched from a parameter server per batch)
* Asynchronous and multithreaded data pipelines
* A highly optimized data loader.
* Supported data formats such as parquet and binary
* Integration with Triton Inference Server for deployment to production


### Getting Started

In this example, we will train a neural network with HugeCTR. We will use NVTabular for preprocessing.

#### Preprocessing and Feature Engineering with NVTabular

We use NVTabular to `Categorify` our categorical input columns.

In [65]:
# External dependencies
import os
import gc

import nvtabular as nvt
import cudf
import numpy as np

from os import path
from sklearn.model_selection import train_test_split

from nvtabular.utils import download_file

We define our base directory, containing the data.

In [66]:
# path to store raw and preprocessed data
BASE_DIR = "/model/data/"

In [67]:
!ls /model

CHANGELOG.md	 Resources.md  images		     requirements.txt
CONTRIBUTING.md  bench	       models		     setup.cfg
HowItWorks.md	 conda	       movielens_hugectr     setup.py
LICENSE		 data	       nvtabular	     tests
MANIFEST.in	 docs	       pyproject.toml	     versioneer.py
README.md	 examples      requirements-dev.txt


If the data is not available in the base directory, we will download and unzip the data.

download_file(
    "http://files.grouplens.org/datasets/movielens/ml-25m.zip", os.path.join(BASE_DIR, "ml-25m.zip")
)

## Preparing the dataset with NVTabular

First, we take a look at the movie metadata.

Let's load the movie ratings.

In [ ]:
ratings = cudf.read_csv(os.path.join(BASE_DIR, "ml-25m", "ratings.csv"))
ratings.head()

We drop the timestamp column and split the ratings into training and test dataset. We use a simple random split.

In [ ]:
ratings = ratings.drop("timestamp", axis=1)
train, valid = train_test_split(ratings, test_size=0.2, random_state=42)

In [ ]:
train.head()

We save our train and valid datasets as parquet files on disk, and below we will read them in while initializing the Dataset objects.

In [ ]:
train.to_parquet(BASE_DIR + "train.parquet")
valid.to_parquet(BASE_DIR + "valid.parquet")

In [ ]:
del train
del valid
gc.collect()

Let's define our categorical and label columns. Note that in that example we do not have numerical columns.

In [ ]:
CATEGORICAL_COLUMNS = ["userId", "movieId"]
LABEL_COLUMNS = ["rating"]

Let's add Categorify op for our categorical features, userId, movieId.

In [ ]:
cat_features = CATEGORICAL_COLUMNS >> nvt.ops.Categorify(cat_cache="device")

The ratings are on a scale between 1-5. We want to predict a binary target with 1 are all ratings >=4 and 0 are all ratings <=3. We use the LambdaOp for it.

In [ ]:
ratings = nvt.ColumnGroup(["rating"]) >> (lambda col: (col > 3).astype("int8"))

We can visualize our calculation graph.

In [ ]:
output = cat_features + ratings
(output).graph

We initialize our NVTabular workflow.

In [ ]:
workflow = nvt.Workflow(output)

We initialize NVTabular Datasets, and use the part_size parameter, which defines the size read into GPU-memory at once, in nvt.Dataset.

In [ ]:
train_dataset = nvt.Dataset(BASE_DIR + "train.parquet", part_size="100MB")
valid_dataset = nvt.Dataset(BASE_DIR + "valid.parquet", part_size="100MB")

First, we collect the training dataset statistics.

In [ ]:
%%time

workflow.fit(train_dataset)

This step is slightly different for HugeCTR. HugeCTR expect the categorical input columns as `int64` and continuous/label columns as `float32`  We can define output datatypes for our NVTabular workflow.

In [ ]:
dict_dtypes = {}

for col in CATEGORICAL_COLUMNS:
    dict_dtypes[col] = np.int64

for col in LABEL_COLUMNS:
    dict_dtypes[col] = np.float32

Note: We do not have numerical output columns

In [ ]:
train_dir = os.path.join(BASE_DIR, "train")
valid_dir = os.path.join(BASE_DIR, "valid")

!rm -rf $train_dir && rm -rf $valid_dir
"""
if path.exists(train_dir):
    os.rmdir(train_dir)
if path.exists(valid_dir):
    os.rmdir(valid_dir)
"""

In addition, we need to provide the data schema to the output calls. We need to define which output columns are `categorical`, `continuous` and which is the `label` columns. NVTabular will write metadata files, which HugeCTR requires to load the data and optimize training.

In [ ]:
workflow.transform(train_dataset).to_parquet(
    output_path=BASE_DIR + "train/",
    shuffle=nvt.io.Shuffle.PER_PARTITION,
    cats=CATEGORICAL_COLUMNS,
    labels=LABEL_COLUMNS,
    dtypes=dict_dtypes,
)
workflow.transform(valid_dataset).to_parquet(
    output_path=BASE_DIR + "valid/",
    shuffle=False,
    cats=CATEGORICAL_COLUMNS,
    labels=LABEL_COLUMNS,
    dtypes=dict_dtypes,
)

In [ ]:
import pandas as pd
train_read = pd.read_parquet(BASE_DIR + "train")

In [ ]:
train_read.head()

In [ ]:
sum(train_read['rating'])

In [146]:
workflow.save(os.path.join(BASE_DIR, "workflow"))


## Scaling Accelerated training with HugeCTR

HugeCTR is a deep learning framework dedicated to recommendation systems. It is written in CUDA C++. As HugeCTR optimizes the training in CUDA++, we need to define the training pipeline and model architecture and execute it via the commandline. We will use the Python API, which is similar to Keras models.

HugeCTR has three main components:
* Solver: Specifies various details such as active GPU list, batchsize, and model_file
* Optimizer: Specifies the type of optimizer and its hyperparameters
* Model: Specifies training/evaluation data (and their paths), embeddings, and dense layers. Note that embeddings must precede the dense layers

**Solver**

Let's take a look on the parameter for the `Solver`. We should be familiar from other frameworks for the hyperparameter.

```
solver = hugectr.solver_parser_helper(
- vvgpu: GPU indices used in the training process, which has two levels. For example: [[0,1],[1,2]] indicates that two nodes are used in the first node. GPUs 0 and 1 are used while GPUs 1 and 2 are used for the second node. It is also possible to specify non-continuous GPU indices such as [0, 2, 4, 7]  
- max_iter: Total number of training iterations
- batchsize: Minibatch size used in training
- display: Intervals to print loss on the screen
- eval_interval: Evaluation interval in the unit of training iteration
- max_eval_batches: Maximum number of batches used in evaluation. It is recommended that the number is equal to or bigger than the actual number of bathces in the evaluation dataset.
If max_iter is used, the evaluation happens for max_eval_batches by repeating the evaluation dataset infinitely.
On the other hand, with num_epochs, HugeCTR stops the evaluation if all the evaluation data is consumed    
- batchsize_eval: Maximum number of batches used in evaluation. It is recommended that the number is equal to or
  bigger than the actual number of bathces in the evaluation dataset
- mixed_precision: Enables mixed precision training with the scaler specified here. Only 128,256, 512, and 1024 scalers are supported
)
```

**Optimizer**

The optimizer is the algorithm to update the model parameters. HugeCTR supports the common algorithms.


```
optimizer = CreateOptimizer(
- optimizer_type: Optimizer algorithm - Adam, MomentumSGD, Nesterov, and SGD 
- learning_rate: Learning Rate for optimizer
)
```

**Model**

We initialize the model with the solver and optimizer:

```
model = hugectr.Model(solver, optimizer)
```

We can add multiple layers to the model with `model.add` function. We will focus on:
- `Input` defines the input data
- `SparseEmbedding` defines the embedding layer
- `DenseLayer` defines dense layers, such as fully connected, ReLU, BatchNorm, etc.

**HugeCTR organizes the layers by names. For each layer, we define the input and output names.**

Input layer:

This layer is required to define the input data.

```
hugectr.Input(
    data_reader_type: Data format to read
    source: The training dataset file list.
    eval_source: The evaluation dataset file list.
    check_type: The data error detection machanism (Sum: Checksum, None: no detection).
    label_dim: Number of label columns
    label_name: Name of label columns in network architecture
    dense_dim: Number of continous columns
    dense_name: Name of contiunous columns in network architecture
    slot_size_array: The list of categorical feature cardinalities
    data_reader_sparse_param_array: Configuration how to read sparse data
    sparse_names: Name of sparse/categorical columns in network architecture
)
```

SparseEmbedding:

This layer defines embedding table

```
hugectr.SparseEmbedding(
    embedding_type: Different embedding options to distribute embedding tables 
    max_vocabulary_size_per_gpu: Maximum vocabulary size or cardinality across all the input features
    embedding_vec_size: Embedding vector size
    combiner: Intra-slot reduction op (0=sum, 1=average)
    sparse_embedding_name: Layer name
    bottom_name: Input layer names
)
```

DenseLayer:

This layer is copied to each GPU and is normally used for the MLP tower.

```
hugectr.DenseLayer(
    layer_type: Layer type, such as FullyConnected, Reshape, Concat, Loss, BatchNorm, etc.
    bottom_names: Input layer names
    top_names: Layer name
    ...: Depending on the layer type additional parameter can be defined
)
```

## Let's define our model

We walked through the documentation, but it is useful to understand the API. Finally, we can define our model. We will write the model to `./model.py` and execute it afterwards.

We need the cardinalities of each categorical feature to assign as `slot_size_array` in the model below.

In [ ]:
from nvtabular.ops import get_embedding_sizes

embeddings = get_embedding_sizes(workflow)
print(embeddings)

In addition, we need the total cardinalities to be assigned as `max_vocabulary_size_per_gpu` parameter.

In [ ]:
total_cardinality = embeddings["movieId"][0] + embeddings["userId"][0]
total_cardinality

In [116]:
%%writefile './model.py'

import hugectr
from mpi4py import MPI

solver = hugectr.solver_parser_helper(
    vvgpu=[[0]],
    max_iter=50000,
    batchsize=2048,
    display=1000,
    eval_interval=1000,
    batchsize_eval=2048,
    max_eval_batches=160,
    i64_input_key=True,
    use_mixed_precision=False,
    repeat_dataset=True,
    snapshot=9999,
)
"""
optimizer = hugectr.optimizer.CreateOptimizer(
    optimizer_type=hugectr.Optimizer_t.Adam, use_mixed_precision=False, learning_rate=1e-3
)
model = hugectr.Model(solver, optimizer)

"""
optimizer = hugectr.optimizer.CreateOptimizer(
    optimizer_type=hugectr.Optimizer_t.SGD, use_mixed_precision=False, learning_rate=1e0
)
model = hugectr.Model(solver, optimizer)

model.add(
    hugectr.Input(
        data_reader_type=hugectr.DataReaderType_t.Parquet,
        source="/model/data/train/_file_list.txt",
        eval_source="/model/data/valid/_file_list.txt",
        check_type=hugectr.Check_t.Non,
        label_dim=1,
        label_name="label",
        dense_dim=0,
        dense_name="dense",
        slot_size_array=[162542, 56586],
        data_reader_sparse_param_array=[
            hugectr.DataReaderSparseParam(hugectr.DataReaderSparse_t.Distributed, 2, 1, 2)
        ],
        sparse_names=["data1"],
    )
)
model.add(
    hugectr.SparseEmbedding(
        embedding_type=hugectr.Embedding_t.LocalizedSlotSparseEmbeddingHash,
        max_vocabulary_size_per_gpu=219128,
        embedding_vec_size=16,
        combiner=0,
        sparse_embedding_name="sparse_embedding1",
        bottom_name="data1",
    )
)
model.add(
    hugectr.DenseLayer(
        layer_type=hugectr.Layer_t.Reshape,
        bottom_names=["sparse_embedding1"],
        top_names=["reshape1"],
        leading_dim=32,
    )
)
model.add(
    hugectr.DenseLayer(
        layer_type=hugectr.Layer_t.InnerProduct,
        bottom_names=["reshape1"],
        top_names=["fc1"],
        num_output=128,
    )
)
model.add(
    hugectr.DenseLayer(
        layer_type=hugectr.Layer_t.ReLU,
        bottom_names=["fc1"],
        top_names=["relu1"],
    )
)
model.add(
    hugectr.DenseLayer(
        layer_type=hugectr.Layer_t.InnerProduct,
        bottom_names=["relu1"],
        top_names=["fc2"],
        num_output=128,
    )
)
model.add(
    hugectr.DenseLayer(
        layer_type=hugectr.Layer_t.ReLU,
        bottom_names=["fc2"],
        top_names=["relu2"],
    )
)
model.add(
    hugectr.DenseLayer(
        layer_type=hugectr.Layer_t.InnerProduct,
        bottom_names=["relu2"],
        top_names=["fc3"],
        num_output=1,
    )
)
model.add(
    hugectr.DenseLayer(
        layer_type=hugectr.Layer_t.BinaryCrossEntropyLoss,
        bottom_names=["fc3", "label"],
        top_names=["loss"],
    )
)
model.compile()
model.summary()
model.fit()

Overwriting ./model.py


In [132]:
!python model.py

===================================Model Init====================================
[06d05h14m19s][HUGECTR][INFO]: Global seed is 3867223216
[06d05h14m21s][HUGECTR][INFO]: Peer-to-peer access cannot be fully enabled.
Device 0: Quadro GV100
[06d05h14m21s][HUGECTR][INFO]: num of DataReader workers: 1
[06d05h14m21s][HUGECTR][INFO]: num_internal_buffers 1
[06d05h14m21s][HUGECTR][INFO]: num_internal_buffers 1
[06d05h14m21s][HUGECTR][INFO]: Vocabulary size: 219128
[06d05h14m21s][HUGECTR][INFO]: max_vocabulary_size_per_gpu_=219128
[06d05h14m21s][HUGECTR][INFO]: All2All Warmup Start
[06d05h14m21s][HUGECTR][INFO]: All2All Warmup End
[06d05h14m22s][HUGECTR][INFO]: gpu0 start to init embedding
[06d05h14m22s][HUGECTR][INFO]: gpu0 init embedding done
==================================Model Summary==================================
Label Name                    Dense Name                    Sparse Name                   
label                         dense                         data1                

We trained our model.

After training terminates, we can see that two `.model` files are generated. We need to move them inside `1` folder under the `movielens_hugectr` folder. Let's create these folders first.

In [ ]:
!mkdir -p /model/movielens_hugectr/1

Now we move our saved `.model` files inside `1` folder.

In [133]:
!mv *.model /model/movielens_hugectr/1/

Note that these stored `.model` files will be used in the inference. Now we have to create a JSON file for inference which has a similar configuration as our training file. We should remove the solver and optimizer clauses and add the inference clause in the JSON file. The paths of the stored dense model and sparse model(s) should be specified at dense_model_file and sparse_model_file within the inference clause. We need to make some modifications to data in the layers clause. Besides, we need to change the last layer from BinaryCrossEntropyLoss to Sigmoid. The rest of "layers" should be exactly the same as that in the training model.py file.

Now let's create a `movielens.json` file inside the `movielens/1` folder. We have already retrieved the cardinality of each categorical column using `get_embedding_sizes` function above. We will use these cardinalities below in the `movielens.json` file as well.

In [89]:
!cat "/model/data/train/_metadata.json"

{"file_stats": [{"file_name": "0.e7063f6295174c2d84c517d37b333719.parquet", "num_rows": 20000076}], "cats": [{"col_name": "userId", "index": 0}, {"col_name": "movieId", "index": 1}], "conts": [], "labels": [{"col_name": "rating", "index": 2}]}

In [90]:
%%writefile '/model/movielens_hugectr/1/movielens.json'

{
   "inference": {
    "max_batchsize": 64,
    "hit_rate_threshold": 0.6,
    "dense_model_file": "/model/models/movielens/1/_dense_1900.model",
    "sparse_model_file": "/model/models/movielens/1/0_sparse_1900.model",
    "label": 1,
    "input_key_type": "I64"
  },
  "layers": [
    {
      "name": "data",
      "type": "Data",
      "format": "Parquet",
      "slot_size_array": [162542, 56586],
      "source": "/model/data/train/_file_list.txt",
      "eval_source": "/model/data/valid/_file_list.txt",
      "check": "Sum",
      "label": {
        "top": "label",
        "label_dim": 1
      },
      "dense": {
        "top": "dense",
        "dense_dim": 0
      },
      "sparse": [
        {
          "top": "data1",
          "type": "DistributedSlot",
          "max_feature_num_per_sample": 3,
          "slot_num": 2
        }
      ]
    },
    {
      "name": "sparse_embedding1",
      "type": "LocalizedSlotSparseEmbeddingHash",
      "bottom": "data1",
      "top": "sparse_embedding1",
      "sparse_embedding_hparam": {
        "max_vocabulary_size_per_gpu": 219128,
        "embedding_vec_size": 16,
        "combiner": 0
      }
    },
    {
      "name": "reshape1",
      "type": "Reshape",
      "bottom": "sparse_embedding1",
      "top": "reshape1",
      "leading_dim": 32
    },
    {
      "name": "fc1",
      "type": "InnerProduct",
      "bottom": "reshape1",
      "top": "fc1",
      "fc_param": {
        "num_output": 128
      }
    },
    {
      "name": "relu1",
      "type": "ReLU",
      "bottom": "fc1",
      "top": "relu1"
    },
    {
      "name": "fc2",
      "type": "InnerProduct",
      "bottom": "relu1",
      "top": "fc2",
      "fc_param": {
        "num_output": 128
      }
    },
    {
      "name": "relu2",
      "type": "ReLU",
      "bottom": "fc2",
      "top": "relu2"
    },
    {
      "name": "fc3",
      "type": "InnerProduct",
      "bottom": "relu2",
      "top": "fc3",
      "fc_param": {
        "num_output": 1
      }
    },
    {
      "name": "sigmoid",
      "type": "Sigmoid",
      "bottom": "fc3",
      "top": "sigmoid"
    } 
  ]
}

Overwriting /model/movielens_hugectr/1/movielens.json


In [91]:
import json
json.load(open("/model/movielens_hugectr/1/movielens.json",'rt'))

{'inference': {'max_batchsize': 64,
  'hit_rate_threshold': 0.6,
  'dense_model_file': '/model/models/movielens/1/_dense_1900.model',
  'sparse_model_file': '/model/models/movielens/1/0_sparse_1900.model',
  'label': 1,
  'input_key_type': 'I64'},
 'layers': [{'name': 'data',
   'type': 'Data',
   'format': 'Parquet',
   'slot_size_array': [162542, 56586],
   'source': '/model/data/train/_file_list.txt',
   'eval_source': '/model/data/valid/_file_list.txt',
   'check': 'Sum',
   'label': {'top': 'label', 'label_dim': 1},
   'dense': {'top': 'dense', 'dense_dim': 0},
   'sparse': [{'top': 'data1',
     'type': 'DistributedSlot',
     'max_feature_num_per_sample': 3,
     'slot_num': 2}]},
  {'name': 'sparse_embedding1',
   'type': 'DistributedSlotSparseEmbeddingHash',
   'bottom': 'data1',
   'top': 'sparse_embedding1',
   'sparse_embedding_hparam': {'max_vocabulary_size_per_gpu': 219128,
    'embedding_vec_size': 16,
    'combiner': 0}},
  {'name': 'reshape1',
   'type': 'Reshape',
   

Now we can save our models to be deployed at the inference stage. To do so we will use `export_hugectr_ensemble` method below. With this method, we can generate the `config.pbtxt` files automatically for each model. In doing so, we should also create a `hugectr_params` dictionary, and define the parameters  like where the `movielens.json` file will be read, `slots` which corresponds to number of categorical features, `embedding_vector_size`, `max_nnz`, and `n_outputs` which is number of outputs.

The script below creates an ensemble triton server model where  

- `workflow` is the the nvtabular workflow used in preprocessing, 
- `hugectr_model_path` is the HugeCTR model that should be served. This path includes the `.model` files.
- `name` is the base name of the various triton models
- `output_path` is the path where is model will be saved to.

In [ ]:
from nvtabular.inference.triton import export_hugectr_ensemble

hugectr_params = dict()
hugectr_params["config"] = "/model/models/movielens/1/movielens.json"
hugectr_params["slots"] = 2
hugectr_params["max_nnz"] = 2
hugectr_params["embedding_vector_size"] = 16
hugectr_params["n_outputs"] = 1
export_hugectr_ensemble(
    workflow=workflow,
    hugectr_model_path="/model/movielens_hugectr/1/",
    hugectr_params=hugectr_params,
    name="movielens",
    output_path="/model/models/",
    label_columns=["rating"],
    cats=CATEGORICAL_COLUMNS,
    max_batch_size=64,
)

After we run the script above, we will have three model folders saved as `movielens_nvt`, `movielens` and `movielens_ens`. Now we can move to the next notebook, `movielens-HugeCTR-inference`, to send request to the Triton Inference Server using the saved ensemble model.

# Reading item embedding

In [119]:
import struct 
import pickle
import numpy as np

key_type = 'I64' # {'I64', 'I32'}, default is 'I32'
key_type_map = {"I32": ["I", 4], "I64": ["q", 8]}

embedding_vec_size = 16

HUGE_CTR_VERSION = 3.0 # set HugeCTR version here, 2.2 for v2.2, 2.21 for v2.21

if HUGE_CTR_VERSION <= 2.2:
    each_key_size = key_type_map[key_type][1] + key_type_map[key_type][1] + 4 * embedding_vec_size
else:
    each_key_size = key_type_map[key_type][1] + 8 + 4 * embedding_vec_size
    
#each_key_size =  key_type_map[key_type][1] + 4 * embedding_vec_size #when using LocalizedSlotSparseEmbeddingHash
 

In [120]:
each_key_size

80

In [121]:
slot_size_array=[56586, 162542]
import numpy as np
np.sum(slot_size_array)

219128

In [97]:
offset = np.insert(np.cumsum(slot_size_array), 0, 0)[:-1]
offset

array([    0, 56586])

In [99]:
embedding_table = [{},{}]

In [153]:
key_count = 0
with open('/model/movielens_hugectr/1/0_sparse_49995.model', 'rb') as file:
        while True:
            buffer = file.read(each_key_size)
            if len(buffer) == 0:
                break
            if HUGE_CTR_VERSION <= 2.2:
                key, slot_id = struct.unpack("2" + key_type_map[key_type][0], 
                                             buffer[0: 2*key_type_map[key_type][1]])
                values = struct.unpack(str(embedding_vec_size) + "f", buffer[2*key_type_map[key_type][1]: ])
            else:
                key = struct.unpack(key_type_map[key_type][0], buffer[0 : key_type_map[key_type][1]])[0]
                slot_id = struct.unpack("Q", buffer[key_type_map[key_type][1] : key_type_map[key_type][1] + 8])[0]
                values = struct.unpack(str(embedding_vec_size) + "f", buffer[key_type_map[key_type][1] + 8: ])
            
            embedding_table[slot_id][key] = values
            key_count +=1
    
print(key_count)

219126


In [154]:
slot_id

1

In [155]:
key

183222

In [156]:
file_size = np.sum(slot_size_array)*each_key_size
file_size

17530240

In [157]:
key_count*each_key_size

17530080

In [158]:
import os
os.path.getsize('//model/movielens_hugectr/1/0_sparse_9999.model')

17530080

In [159]:
for i in range(2): 
    print(len(embedding_table[i].keys()))

162541
56585


In [160]:
# Extracting item embedding
item_embedding = 1e99 * np.ones((56586, embedding_vec_size), dtype='float')
for i in embedding_table[-1].keys():
    item_embedding[i-162542] = embedding_table[-1][i]



In [161]:
import pandas as pd

movies_mapping = pd.read_parquet(BASE_DIR + '/workflow/categories/unique.movieId.parquet')

In [162]:
nn_to_movies = {}
movies_to_nn = {}
for i, row in movies_mapping.iterrows():
    movies_to_nn[row.movieId] = i
    nn_to_movies[i] = row.movieId

import pandas as pd
movies = pd.read_csv("./data/ml-25m/movies.csv", index_col="movieId")

In [163]:
from scipy.spatial.distance import cdist

def find_similar_movies(nn_movie_id, item_embedding, k=10, metric="euclidean"):
    #find the top K similar items according to one of the distance metric: cosine or euclidean
    sim = 1-cdist(item_embedding, item_embedding[nn_movie_id].reshape(1, -1), metric=metric)
   
    return sim.squeeze().argsort()[-k:][::-1]

In [164]:
for movie_ID in range(1,100):
    if movie_ID in movies.index:    
        print("Query: ", movies.loc[movie_ID]["title"], movies.loc[movie_ID]["genres"])

        print("Similar movies: ")
        similar_movies = find_similar_movies(movies_to_nn[movie_ID], item_embedding)

        for i in similar_movies:
            if i ==0: #missing movie
                continue
            print(nn_to_movies[i], movies.loc[nn_to_movies[i]]["title"], movies.loc[nn_to_movies[i]]["genres"])
        print("=================================\n")
    

Query:  Toy Story (1995) Adventure|Animation|Children|Comedy|Fantasy
Similar movies: 
1.0 Toy Story (1995) Adventure|Animation|Children|Comedy|Fantasy
1210.0 Star Wars: Episode VI - Return of the Jedi (1983) Action|Adventure|Sci-Fi
8950.0 The Machinist (2004) Drama|Mystery|Thriller
5244.0 Shogun Assassin (1980) Action|Adventure
56015.0 Kansas City Confidential (1952) Crime|Drama|Film-Noir|Mystery
6983.0 Jane Eyre (1944) Drama|Romance
60291.0 Gonzo: The Life and Work of Dr. Hunter S. Thompson (2008) Documentary
175791.0 LA 92 (2017) Documentary
70286.0 District 9 (2009) Mystery|Sci-Fi|Thriller
5464.0 Road to Perdition (2002) Crime|Drama

Query:  Jumanji (1995) Adventure|Children|Fantasy
Similar movies: 
2.0 Jumanji (1995) Adventure|Children|Fantasy
329.0 Star Trek: Generations (1994) Adventure|Drama|Sci-Fi
131457.0 Rise of the Legend (2014) Action|Drama
118706.0 Black Sea (2015) Adventure|Mystery|Thriller
140133.0 Hollywood Chainsaw Hookers (1988) Comedy|Horror
33085.0 Amityville Horror